In [4]:
import pandas as pd

In [2]:
def merge_chunks(right_on_list, input_file, output_file, rename_columns={}, drop_columns=[], chunksize=10 ** 7, how='left'):
    first_time = True
    f = open(output_file, 'w')
    size = 0

    for left in pd.read_csv(input_file, chunksize=chunksize):
        for right, on in right_on_list:
            left = left.merge(right, how=how, on=on)
            
        if len(rename_columns)>0:
            left.rename(columns=rename_columns, inplace=True)
            
        if len(drop_columns)>0:
            left.drop(drop_columns, axis=1, inplace = True)
    
        if first_time:
            left.to_csv(f, index = False, header=True)
            f.close()
            f = open(output_file, 'a')
            first_time = False
            left.head()
        else:
            left.to_csv(f, index = False, header=False)
        
        size+=chunksize
        print size
            
    f.close()
    print 'Done'

### Load tables to be joined

In [3]:
%run "tools.py"

In [4]:
pd.read_csv("../download/clicks_train.csv", nrows=10).head()

,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0
2,1,144739,1
3,1,156824,0
4,1,279295,0


In [5]:
events = pd.read_csv("../generated/events.csv", usecols=['display_id', 'document_id', 'timestamp', 'day'])
events.head()

,display_id,document_id,timestamp,day
0,1,379743,61,14
1,2,1794259,81,14
2,3,1179111,182,14
3,4,1777797,234,14
4,5,252458,338,14


In [6]:
adsPerDisplay = normilize(pd.read_csv("../generated/adsPerDisplay.csv"))
adsPerDisplay.head()

,display_id,adsPerDisplay
0,1,0.454491
1,2,-0.631378
2,3,0.454491
3,4,-0.088444
4,5,-0.088444


In [7]:
#to be used with Event documents only as page_rate
documentShows = normilize(pd.read_csv("../generated/page_views_event_docs_size.csv"))
documentShows.head()

,document_id,size
0,2,0.501183
1,4,-0.055494
2,5,-0.035036
3,7,-0.052128
4,8,-0.042402


In [8]:
docs_content = normilize(pd.read_csv("../generated/documents_content.csv"), exclude=['published'])
docs_content.head()

,document_id,published,source_rate,publisher_rate,topics_rate,categories_rate
0,379743,-2147483648,-0.214989,-0.456605,-0.910963,-1.014116
1,1794259,-14399998,0.004755,-0.236996,-0.826177,0.250996
2,1179111,-2147483648,-0.272270,-0.586902,-0.029274,0.031288
3,1777797,-2147483648,0.111585,-0.290756,-0.244900,-0.784794
4,252458,-2147483648,0.004755,-0.236996,0.473743,2.283626


In [9]:
clicksPerAd_train = normilize(pd.read_csv("../generated/clicks_shows_PerAd.csv", usecols=['ad_id', 'clicksPerShows']))
clicksPerAd_train.head()

,ad_id,clicksPerShows
0,1,-0.657944
1,2,-0.449135
2,3,-0.344082
3,4,-0.083718
4,5,-0.657944


### Joining: Step 1

In [10]:
right_on_list = [(events, 'display_id'), (adsPerDisplay, 'display_id'),
                 (documentShows, 'document_id'), (docs_content, 'document_id'),
                 (clicksPerAd_train, 'ad_id')]

In [11]:
rename_columns = {'size':'page_rate', 'published':'page_published', 'source_rate':'page_source_rate',
                  'publisher_rate':'page_publisher_rate', 'topics_rate':'page_topics_rate', 'categories_rate':'page_cat_rate',
                  'clicksPerShows':'ad_click_rate'}

In [12]:
drop_columns = ['document_id']

In [22]:
input_file = "../download/clicks_train.csv"
output_file = "../temp/clicks_train_step1.csv"

In [14]:
merge_chunks(right_on_list, input_file, output_file, rename_columns, drop_columns)

10000000
20000000
30000000
40000000
50000000
60000000
70000000
80000000
90000000
Done


In [23]:
pd.read_csv(output_file, nrows=10).head()

,display_id,ad_id,clicked,timestamp,day,adsPerDisplay,page_rate,page_published,page_source_rate,page_publisher_rate,page_topics_rate,page_cat_rate,ad_rate
0,1,42337,0,61,14,0.454491,0.005304,-2147483648,-0.214989,-0.456605,-0.910963,-1.014116,0.110486
1,1,139684,0,61,14,0.454491,0.005304,-2147483648,-0.214989,-0.456605,-0.910963,-1.014116,0.103228
2,1,144739,1,61,14,0.454491,0.005304,-2147483648,-0.214989,-0.456605,-0.910963,-1.014116,0.831875
3,1,156824,0,61,14,0.454491,0.005304,-2147483648,-0.214989,-0.456605,-0.910963,-1.014116,-0.508816
4,1,279295,0,61,14,0.454491,0.005304,-2147483648,-0.214989,-0.456605,-0.910963,-1.014116,0.836758


In [28]:
del events

### Joining: Step 2

In [16]:
ads = normilize(pd.read_csv("../generated/promoted_content.csv"))
ads.head()

,ad_id,document_id,doc_rate,campaign_rate,advertiser_rate
0,1,6614,-0.343316,-0.433721,-0.803192
1,6,12736,-0.343316,-0.433721,-0.803192
2,7,12808,-0.343316,-0.433721,-0.803192
3,9,13379,-0.343316,-0.433721,-0.803192
4,10,13885,-0.343316,-0.433721,-0.803192


In [24]:
right_on_list = [(ads, 'ad_id'), (docs_content, 'document_id')]
rename_columns = {'doc_rate':'addoc_rate', 'categories_rate':'addoc_cat_rate',
                  'published':'addoc_published', 'source_rate':'addoc_source_rate',
                  'publisher_rate':'addoc_publisher_rate', 'topics_rate':'addoc_topics_rate'}
drop_columns = ['document_id']    
input_file = "../temp/clicks_train_step1.csv"
output_file = "../temp/clicks_train_step2.csv"

In [26]:
merge_chunks(right_on_list, input_file, output_file, rename_columns, drop_columns)

10000000
20000000
30000000
40000000
50000000
60000000
70000000
80000000
90000000
Done


In [27]:
pd.read_csv(output_file, nrows=10).head()

,display_id,ad_id,clicked,timestamp,day,adsPerDisplay,page_rate,page_published,page_source_rate,page_publisher_rate,...,page_cat_rate,ad_rate,addoc_rate,campaign_rate,advertiser_rate,addoc_published,addoc_source_rate,addoc_publisher_rate,addoc_topics_rate,addoc_cat_rate
0,1,42337,0,61,14,0.454491,0.005304,-2147483648,-0.214989,-0.456605,...,-1.014116,0.110486,-0.341203,-0.478144,-0.818183,-1137599998,-0.272215,-0.605843,-0.819885,-0.222556
1,1,139684,0,61,14,0.454491,0.005304,-2147483648,-0.214989,-0.456605,...,-1.014116,0.103228,-0.329048,-0.475183,-0.817748,-2147483648,-0.272270,-0.605887,-0.336479,-0.297341
2,1,144739,1,61,14,0.454491,0.005304,-2147483648,-0.214989,-0.456605,...,-1.014116,0.831875,-0.341203,-0.472221,-0.822746,-2147483648,-0.272270,-0.605887,0.303982,-0.842784
3,1,156824,0,61,14,0.454491,0.005304,-2147483648,-0.214989,-0.456605,...,-1.014116,-0.508816,-0.339089,-0.461856,-0.801888,-2147483648,-0.271756,-0.605466,-0.680455,-0.824454
4,1,279295,0,61,14,0.454491,0.005304,-2147483648,-0.214989,-0.456605,...,-1.014116,0.836758,-0.333276,-0.450010,-0.794718,-1040399998,-0.271972,-0.605643,-0.492251,0.273409


### Calculate additional columns

In [32]:
def calc_chunks(input_file, output_file, drop_columns=[], chunksize=10000000):
    first_time = True
    f = open(output_file, 'w')
    size = 0

    for chunk in pd.read_csv(input_file, chunksize=chunksize):
        chunk['page_age'] = chunk['timestamp'] - chunk['page_published']
        chunk['addoc_age'] = chunk['timestamp'] - chunk['addoc_published']
        
        if len(drop_columns)>0:
            chunk.drop(drop_columns, axis=1, inplace = True)
    
        if first_time:
            chunk.to_csv(f, index = False, header=True)
            f.close()
            f = open(output_file, 'a')
            first_time = False
            chunk.head()
        else:
            chunk.to_csv(f, index = False, header=False)
        
        size+=chunksize
        print size
            
    f.close()
    print 'Done'    

In [35]:
#usecols=['display_id', 'ad_id', 'timestamp', 'clicked', 'doc_published', 'doc_topics_rate', 'published', 'topics_rate']
input_file = "../temp/clicks_train_step2.csv"
output_file = "../temp/clicks_train_step3.csv"
drop_columns = ['timestamp', 'page_published', 'addoc_published']

In [36]:
calc_chunks(input_file, output_file, drop_columns=drop_columns)

10000000
20000000
30000000
40000000
50000000
60000000
70000000
80000000
90000000
Done


In [38]:
pd.read_csv(output_file, nrows=10).head()

,display_id,ad_id,clicked,day,adsPerDisplay,page_rate,page_source_rate,page_publisher_rate,page_topics_rate,page_cat_rate,ad_rate,addoc_rate,campaign_rate,advertiser_rate,addoc_source_rate,addoc_publisher_rate,addoc_topics_rate,addoc_cat_rate,page_age,addoc_age
0,1,42337,0,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,0.110486,-0.341203,-0.478144,-0.818183,-0.272215,-0.605843,-0.819885,-0.222556,2147483709,1137600059
1,1,139684,0,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,0.103228,-0.329048,-0.475183,-0.817748,-0.272270,-0.605887,-0.336479,-0.297341,2147483709,2147483709
2,1,144739,1,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,0.831875,-0.341203,-0.472221,-0.822746,-0.272270,-0.605887,0.303982,-0.842784,2147483709,2147483709
3,1,156824,0,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,-0.508816,-0.339089,-0.461856,-0.801888,-0.271756,-0.605466,-0.680455,-0.824454,2147483709,2147483709
4,1,279295,0,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,0.836758,-0.333276,-0.450010,-0.794718,-0.271972,-0.605643,-0.492251,0.273409,2147483709,1040400059


### Normalization of additional columns

In [7]:
import pandas as pd

In [8]:
from sklearn.preprocessing import StandardScaler

def norm_chunks(input_file, output_file, usecols, chunksize=1000000):
    print "Start..."
    data = pd.read_csv(input_file, usecols=usecols)
    print "Columns are loaded. Start fitting..."
    fits = {}
    for col in usecols:
        fits[col] = StandardScaler().fit(data[[col]])
    del data
    
    print "Normalizing..."
    first_time = True
    f = open(output_file, 'w')
    size = 0

    for chunk in pd.read_csv(input_file, chunksize=chunksize):
        for col in usecols:
            chunk[col] = fits[col].transform(chunk[[col]])
    
        if first_time:
            chunk.to_csv(f, index = False, header=True)
            f.close()
            f = open(output_file, 'a')
            first_time = False
            chunk.head()
        else:
            chunk.to_csv(f, index = False, header=False)
        
        size+=chunksize
        print size
            
    f.close()
    print 'Done'    

In [5]:
input_file = "../temp/clicks_train_step3.csv"
output_file = "../generated/clicks_train_final.csv"
usecols=['page_age', 'addoc_age']

In [2]:
norm_chunks(input_file, output_file, usecols=usecols)

NameError: name 'norm_chunks' is not defined

In [6]:
pd.read_csv(output_file, nrows=10).head()

,display_id,ad_id,clicked,day,adsPerDisplay,page_rate,page_source_rate,page_publisher_rate,page_topics_rate,page_cat_rate,ad_rate,addoc_rate,campaign_rate,advertiser_rate,addoc_source_rate,addoc_publisher_rate,addoc_topics_rate,addoc_cat_rate,page_age,addoc_age
0,1,42337,0,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,0.110486,-0.341203,-0.478144,-0.818183,-0.272215,-0.605843,-0.819885,-0.222556,0.727377,-1.767554
1,1,139684,0,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,0.103228,-0.329048,-0.475183,-0.817748,-0.272270,-0.605887,-0.336479,-0.297341,0.727377,-0.392610
2,1,144739,1,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,0.831875,-0.341203,-0.472221,-0.822746,-0.272270,-0.605887,0.303982,-0.842784,0.727377,-0.392610
3,1,156824,0,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,-0.508816,-0.339089,-0.461856,-0.801888,-0.271756,-0.605466,-0.680455,-0.824454,0.727377,-0.392610
4,1,279295,0,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,0.836758,-0.333276,-0.450010,-0.794718,-0.271972,-0.605643,-0.492251,0.273409,0.727377,-1.899891


In [11]:
print pd.read_csv(output_file, nrows=2).columns

Index([u'display_id', u'ad_id', u'clicked', u'day', u'adsPerDisplay',
       u'page_rate', u'page_source_rate', u'page_publisher_rate',
       u'page_topics_rate', u'page_cat_rate', u'ad_rate', u'addoc_rate',
       u'campaign_rate', u'advertiser_rate', u'addoc_source_rate',
       u'addoc_publisher_rate', u'addoc_topics_rate', u'addoc_cat_rate',
       u'page_age', u'addoc_age'],
      dtype='object')


### Pivot

In [17]:
df = pd.read_csv(output_file, nrows=6)

In [18]:
df

,display_id,ad_id,clicked,day,adsPerDisplay,page_rate,page_source_rate,page_publisher_rate,page_topics_rate,page_cat_rate,ad_rate,addoc_rate,campaign_rate,advertiser_rate,addoc_source_rate,addoc_publisher_rate,addoc_topics_rate,addoc_cat_rate,page_age,addoc_age
0,1,42337,0,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,0.110486,-0.341203,-0.478144,-0.818183,-0.272215,-0.605843,-0.819885,-0.222556,0.727377,-1.767554
1,1,139684,0,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,0.103228,-0.329048,-0.475183,-0.817748,-0.272270,-0.605887,-0.336479,-0.297341,0.727377,-0.392610
2,1,144739,1,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,0.831875,-0.341203,-0.472221,-0.822746,-0.272270,-0.605887,0.303982,-0.842784,0.727377,-0.392610
3,1,156824,0,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,-0.508816,-0.339089,-0.461856,-0.801888,-0.271756,-0.605466,-0.680455,-0.824454,0.727377,-0.392610
4,1,279295,0,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,0.836758,-0.333276,-0.450010,-0.794718,-0.271972,-0.605643,-0.492251,0.273409,0.727377,-1.899891
5,1,296965,0,14,0.454491,0.005304,-0.214989,-0.456605,-0.910963,-1.014116,-0.117496,1.905250,-0.334509,1.358825,-0.271891,-0.605577,-0.807861,0.454569,0.727377,-0.392610


In [ ]:
df.pivot(index='display_id', columns='ad_id', values=['ad_id','clicked']).reset_index()

In [22]:
df_not = df[df.clicked==0]
df_clicked = df[df.clicked==1]

In [34]:
titles = df_clicked.mean().values
titles

array([  1.00000000e+00,   1.44739000e+05,   1.00000000e+00,
         1.40000000e+01,   4.54490596e-01,   5.30398472e-03,
        -2.14988638e-01,  -4.56605031e-01,  -9.10963090e-01,
        -1.01411622e+00,   8.31874641e-01,  -3.41202592e-01,
        -4.72221182e-01,  -8.22745577e-01,  -2.72269507e-01,
        -6.05887017e-01,   3.03982482e-01,  -8.42784134e-01,
         7.27376895e-01,  -3.92610363e-01])

In [36]:
compare = pd.DataFrame({'title':df_clicked.mean().index, 'clicked':df_clicked.mean().values, 'not':df_not.mean().values})
compare

,clicked,not,title
0,1.000000,1.000000,display_id
1,144739.000000,183021.000000,ad_id
2,1.000000,0.000000,clicked
3,14.000000,14.000000,day
4,0.454491,0.454491,adsPerDisplay
5,0.005304,0.005304,page_rate
6,-0.214989,-0.214989,page_source_rate
7,-0.456605,-0.456605,page_publisher_rate
8,-0.910963,-0.910963,page_topics_rate
9,-1.014116,-1.014116,page_cat_rate


### To HDF5

In [1]:
import pandas as pd

In [10]:
def writeCSVtoHDF5(input_file, output_file, chunksize = 5000000):
    first_time = True
    size = 0

    for chunk in pd.read_csv(input_file, chunksize=chunksize):
        chunk.to_hdf(output_file, key='train', mode='a', append=True)
        
        size+=chunksize
        print size
            
    print 'Done'    

In [2]:
input_file = "../generated/clicks_train_final.csv"
output_file = "../generated/clicks_train_final.hdf5"

In [ ]:
writeCSVtoHDF5(input_file, output_file)

5000000
10000000
15000000
20000000
25000000
30000000
35000000
40000000
45000000
50000000
55000000
60000000
65000000
70000000
75000000
80000000
85000000
90000000
Done


In [19]:
clicks = pd.HDFStore(output_file, mode='a')  #pd.read_hdf(output_file, nrows=10).head()

<class 'pandas.io.pytables.HDFStore'>
File path: ../generated/clicks_train_final.hdf5
Empty

In [17]:
c = clicks.select_column('train', 'index')
#c.count()

AttributeError: 'NoneType' object has no attribute 'read_column'

### Split

In [ ]:
import pandas as pd
#from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit

In [ ]:
input_file = "../generated/clicks_train_final.hdf5"
output_train = "../generated/clicks_train_final_train.hdf5"
output_test = "../generated/clicks_train_final_test.hdf5"

In [ ]:
clicks = pd.read_hdf(input_file)
clicks.count()

In [ ]:
#kf = KFold(n_splits=5)
#train, test = kf.split(displays).next()
ss = ShuffleSplit(n_splits=1, test_size=0.25, random_state=0)
train_index, test_index = ss.split(clicks).next()

print len(train_index) + len(test_index)

In [ ]:
clicks.loc[train_index].to_hdf(output_train)

In [ ]:
clicks.loc[test_index].to_hdf(output_test)